In [ ]:
import jax.numpy as jnp
import numpy as np
import time
import chex
import jax
import gymnasium as gym
import exciting_environments as excenv
import random
import matplotlib.pyplot as plt
import diffrax

### Simulate Ahead

In [ ]:
env=excenv.MassSpringDamper(batch_size=5,action_constraints={"force"=[10,20,30,40,50]})

In [ ]:
acts=jnp.full((5,8),jnp.array([1,1,1,1,1,1,1,1])*10)[:,:,None]

In [ ]:
state=env.vmap_init_state()
obs,states,last_state=env.vmap_sim_ahead(state,
                   acts,
                   env.tau,
                   env.tau
                   )
obs[0]

In [ ]:
k=[]
def test(x):
    return x.physical_state.deflection

In [ ]:
states

In [ ]:
jax.vmap(test, in_axes=(0))(state)

In [ ]:
env.generate_rew_trunc_term_ahead(states,acts)

In [ ]:
state

In [ ]:
state

In [ ]:
obs.shape

In [ ]:
acts.shape

In [ ]:
state=env.vmap_init_state()
for _ in range(acts.shape[1]):
    obs,state=env.vmap_step(state=state,action=10*jnp.ones(5).reshape(-1,1))
    print(obs[0,:])